Modelando Textos Probabilisticamente
====================================

Nesta prática, vamos usar redes neurais para estimar as probabilidades condicionais de textos, caractere a caractere.
Para uma discussão interessante sobre o assunto, veja o seguinte blog: http://karpathy.github.io/2015/05/21/rnn-effectiveness/.

Vamos usar a biblioteca Keras adaptando um de seus exemplos.

In [56]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.utils.data_utils import get_file
import numpy as np
import scipy.stats as st
import random
import sys

Primeiro vamos utilizar o mesmo texto usado no exemplo original

In [2]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")

In [3]:
try: 
    text = open(path).read().lower()
except UnicodeDecodeError:
    import codecs
    text = codecs.open(path, encoding='utf-8').read().lower()

In [4]:
print('Comprimento do corpus:', len(text))

Comprimento do corpus: 600893


Como o  modelo vai se basear em caracteres, precisamos definir o conjunto de caracteres do texto:

In [5]:
chars = set(text)
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


O modelo envolve probabilidades condicionais entre caracteres consecutivos, então precisamos alimentar o modelo com sequências de caracteres, com sobreposição.

In [6]:
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('num sequences:', len(sentences))

num sequences: 200291


In [7]:
print('Vetorizando...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vetorizando...


In [8]:
print('Construindo o modelo...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Construindo o modelo...


In [63]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.exp(np.log(a) / temperature)
    a /= a.sum() +.001
    try:
        sp = np.argmax(st.multinomial.rvs(1,a,1))
        
#         sp = np.argmax(np.random.multinomial(1, a, 1))
    except ValueError as e:
        print(a[:-1].sum(), len(a),a)
        raise(e)
    return  sp

In [65]:

model.fit(X, y, batch_size=1024, epochs=5)

Epoch 1/5
200291/200291 [==============================] - 87s - loss: 1.4636    
Epoch 2/5
200291/200291 [==============================] - 87s - loss: 1.3965    
Epoch 3/5
200291/200291 [==============================] - 87s - loss: 1.3391    
Epoch 4/5
200291/200291 [==============================] - 87s - loss: 1.2856    
Epoch 5/5
200291/200291 [==============================] - 87s - loss: 1.2323    


In [66]:
    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


----- diversity: 0.2
----- Generating with seed: " distress, and twice"
 distress, and twice the -nothere --who --the ---the ---the ---the --but as the ------                                                                                                                                                                                                                                                                                                                                              

----- diversity: 0.5
----- Generating with seed: " distress, and twice"
 distress, and twice the same as a discreating the artists, -who are selicate instincts of the soul, who has such a grateful and expeciences, the ansters of a tho-soral will to all the problem of the free spirit of the eternal life in the contrariction of the individual with the same with the most depe-denion of the same for the artist in others had the sense of the contrary the whole shall then have been maxe for th

----- diver

In [58]:
st.multinomial.rvs(1,[0.5,0.5,0],1)

array([[0, 1, 0]])